In [1]:
import os
import sys
import re

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

import json
import numpy as np
import pandas as pd

from tqdm import tqdm

# Loading functions from the scripts
"""
Mostafa:
I used the new structured output for question generation.
It's a beta version, but it works on my end (10/23/2024).
https://platform.openai.com/docs/guides/structured-outputs/structured-outputs

For answer generation, I had some issues, so I used the standard API."

Please upgrade before running this notebook: pip install --upgrade openai
"""

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))  # Get parent directory of the notebook 
sys.path.append(parent_dir)  #  to the Python path

from scripts.chunking import process_latex_files
from scripts.embedding import get_embeddings, fixed_knn_retrieval
from scripts.prompts import gen_questions, gen_questions_s, gen_answer


/home/ben/Documents/repos/comp-phys-transformer/scripts/prompts.py:187: SyntaxWarning: invalid escape sequence '\s'
  """
/home/ben/Documents/repos/comp-phys-transformer/scripts/prompts.py:187: SyntaxWarning: invalid escape sequence '\s'
  """
/home/ben/Documents/repos/comp-phys-transformer/scripts/prompts.py:187: SyntaxWarning: invalid escape sequence '\s'
  """
/home/ben/Documents/repos/comp-phys-transformer/scripts/prompts.py:187: SyntaxWarning: invalid escape sequence '\s'
  """
/home/ben/Documents/repos/comp-phys-transformer/scripts/prompts.py:187: SyntaxWarning: invalid escape sequence '\e'
  """


## 0. Setting API and Models

In [2]:
"""
Mostafa: 
I suggest using 'gpt-4o' for production runs, but it is more expensive.
For embeddings, I recommend 'text-embedding-3-large.' We only need to run it once, but it also costs more.

# https://openai.com/api/pricing/
# https://openai.com/index/new-embedding-models-and-api-updates/
# https://platform.openai.com/docs/guides/embeddings/embedding-models
"""

api_key = os.getenv("OPENAI_API_KEY")  # Replace with your actual API key
client = OpenAI(api_key=api_key)

production_mode = True
chunk_by_section = True
chpt_for_quest_answ = 1

if production_mode == False:
    llm_model = "gpt-4o-mini"
    embedding_model = "text-embedding-3-small"    # NOTE: this must be the same for all embeddings. 
elif production_mode == True:
    llm_model = "gpt-4o"
    embedding_model = "text-embedding-3-large"    # NOTE: this must be the same for all embeddings. 


# Setting path for root data folder
main_dir = '../data/hughes_latex_Q_then_A_use_context'

if not os.path.exists(main_dir):
    os.makedirs(main_dir)


## 1. Generating Context Embedding Space

In [3]:
"""
Mostafa: 
I used fixed size chunks (512) with a 25% overlap.
Make sure environment_sensitive is set to False for fixed size.

We should embed all chapters to generate the embedding space. For the demo, I only included two chapters.
please update the paths in latex_file_paths.
"""

# add all book chapters paths
if production_mode == False:
    latex_file_paths = [ 
        '../data/FEM_Hughes_LaTeX_Textbook/chapter1.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter2.tex'
    ]
elif production_mode == True:
    latex_file_paths = [ 
        '../data/FEM_Hughes_LaTeX_Textbook/chapter1.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter2.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter3.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter4.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter5.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter6.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter7.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter8.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter9.tex',
        '../data/FEM_Hughes_LaTeX_Textbook/chapter10.tex'
    ]

tokens_per_chunk = 4096                         # was 512
token_overlap = int(0.25 * tokens_per_chunk)    # 25% overlap
environment_sensitive = False                   # If False, equations can split between two chunks, but chunk lengths remain fixed.

#------------------------------------------------------------------------
if chunk_by_section == False:
    embedding_space_file_name = f'{main_dir}/hughes_latex_embedding_space_tpc{tokens_per_chunk}_o{token_overlap}.json'
elif chunk_by_section == True:
    embedding_space_file_name = f'{main_dir}/hughes_latex_embedding_space_by_sections_tpc{tokens_per_chunk}.json'
    token_overlap = 0

space = {}
if not os.path.exists(embedding_space_file_name):
    
    chunks = process_latex_files(latex_file_paths, 
                                 tokens_per_chunk, 
                                 token_overlap, 
                                 environment_sensitive, 
                                 chunk_by_section = chunk_by_section)
    
    chunk_length = []
    char_length = []
    print(chunks)
    for chunk in chunks:
        print(f"chunk word length: {len(chunk.split(" "))}, chunk char length: {len(chunk)}, chunk = {chunk}")
        chunk_length.append(len(chunk.split(" ")))
        char_length.append(len(chunk))
    print(f"max chunk length in words = {np.max(chunk_length)}")
    print(f"max chunk length in char = {np.max(char_length)}")
    #print(f"chunk lengths = {chunk_length}")

    # using api
    embedding_space = get_embeddings(client, chunks, model=embedding_model)
    
    # save
    with open(embedding_space_file_name, 'w') as json_file:
        json.dump({'chunks': chunks, 'embedding_space': embedding_space}, json_file)

    print("saved")
else:
    # save
    with open(embedding_space_file_name, 'r') as json_file:
        loaded_data = json.load(json_file)

    chunks = loaded_data['chunks']
    embedding_space = np.array(loaded_data['embedding_space'])
    print("loaded")

chunks = np.array(chunks)
embedding_space = np.array(embedding_space)
print("Space size:", embedding_space.shape)


loaded
Space size: (331, 3072)


## 2. Generating Questions and Their Embeddings

In [4]:
"""
Mostafa: 
For generating questions, we want larger chunks with a bit of overlap.
The following values are just for this demo, so please adjust them as needed.

I only ran Chapter One.
"""


chapter = chpt_for_quest_answ
latex_file_path = f'../data/FEM_Hughes_LaTeX_Textbook/chapter{chapter}.tex'

max_questions = 40                             # max number of questions per chunk

tokens_per_chunk = 1536                       
token_overlap = int(0.2 * tokens_per_chunk)   # 10% overlap
environment_sensitive = True                  # If True, equations won't be split between chunks, which may result in chunks larger than the specified tokens_per_chunk
                           

#------------------------------------------------------------------------
if chunk_by_section == False:
    questions_file_name = f"{main_dir}/hughes_ch{chapter}_Qs_n{max_questions}_tpc{tokens_per_chunk}_o{token_overlap}.json"   
elif chunk_by_section == True:
    questions_file_name = f"{main_dir}/hughes_ch{chapter}_Qs_n{max_questions}_by_sections_tpc{tokens_per_chunk}.json"  
    token_overlap = 0 

if not os.path.exists(questions_file_name):
    question_chunks = process_latex_files(latex_file_path, tokens_per_chunk, token_overlap, environment_sensitive, chunk_by_section=chunk_by_section)
    
    if production_mode == False:
        question_chunks = question_chunks[1:6] # for testing small batch
    
    for question in question_chunks:
        print(f"chunk word length: {len(question.split(" "))}, chunk char length: {len(question)}, chunk = {question}")

    questions = {}  # main data

    # we should save generation info we used
    questions['info'] = {
        'tokens_per_chunk': tokens_per_chunk,
        'token_overlap': token_overlap,
        'environment_sensitive': environment_sensitive,
        'max_questions': max_questions
    }

    ## step 1: generate questions
    questions['data'] = []
    for i in tqdm(range(len(question_chunks)), desc="Generating Questions"):
        # q_for_chunk = gen_questions(client, question_chunks[i], max_questions, model=llm_model)
        q_for_chunk = gen_questions_s(client, question_chunks[i], max_questions, model=llm_model)   # Using the new function
        questions['data'].append({'chunk': question_chunks[i],'questions': q_for_chunk})
    print('Questions are generated')

    ## step 2: embedding all questions at once
    all_questions = []
    for item in questions['data']:
        for sub_item in item['questions']:
            all_questions.append(sub_item['question'])
    # using api
    embeddings = get_embeddings(client, all_questions, model=embedding_model) 
    # add them to data:
    k = 0
    for item in questions['data']:
        for sub_item in item['questions']:
            sub_item['embedding'] = embeddings[k]
            k +=1
    print('Questions are embedded')
    

    with open(questions_file_name, 'w') as json_file:
        json.dump(questions, json_file, indent=4)
    print('saved', questions_file_name)

else:
    with open(questions_file_name, 'r') as json_file:
        questions = json.load(json_file)

    print('loaded', questions_file_name)


chunk word length: 1, chunk char length: 10, chunk = \maketitle
chunk word length: 525, chunk char length: 3363, chunk = The main constituents of a finite element method for the solution of a boundary-value problem are\\ i. The variational or weak statement of the problem; and\\ ii. The approximate solution of the variational equations through the use of "finite element functions." To clarify concepts we shall begin with a simple example.\\ Suppose we want to solve the following differential equation for $u$ : \begin{equation*} u_{, x x}+f=0 \tag{1.1.1} \end{equation*} where a comma stands for differentiation (i.e., $u_{, x x}=d^{2} u / d x^{2}$ ). We assume $f$ is a given smooth, scalar-valued function defined on the unit interval. We write \begin{equation*} f: [0,1] \to \mathbb{R} \tag{1.1.2} \end{equation*} where $[0,1]$ stands for the unit interval (i.e., the set of points $x$ such that $0 \leq x \leq 1$ ) and $\mathbb{R}$ stands for the real numbers. In words, (1.1.2) states that 

Generating Questions: 100%|██████████| 34/34 [05:55<00:00, 10.46s/it]


Questions are generated
Questions are embedded
saved ../data/hughes_latex_Q_then_A_use_context/hughes_ch1_Qs_n40_by_sections_tpc1536.json


## 3. Context Retrieval and Generating Answers 

In [5]:
"""
Mostafa: 
Since we answer each question separately, this process is slow.
We might want to consider using the batch API for this.
"""

top_k = 10   # number of retrieved closest contexts         

#------------------------------------------------------------------------
if chunk_by_section == False:
    questions_answers_file_name = f"{main_dir}/hughes_ch{chapter}_QAs_n{max_questions}_topk{top_k}_tpc{tokens_per_chunk}_o{token_overlap}.json"   
elif chunk_by_section == True:
    questions_answers_file_name = f"{main_dir}/hughes_ch{chapter}_QAs_n{max_questions}_topk{top_k}_by_sections.json"   

if not os.path.exists(questions_answers_file_name):

    questions_answers = questions.copy()

    # step 1) finding top_k context from the book embedding and adding them to each question
    for item in questions_answers['data']:
        for sub_item in item['questions']:
            ind = fixed_knn_retrieval(sub_item['embedding'], embedding_space, top_k)
            context = ''
            for i, chunk in enumerate(chunks[ind]):
                context += f'\n\n Additional context {i}: {chunk}' 
            sub_item['context'] = context
    print('top_k context added')

    # step 2) generating answers (slow)  (should we try batch API?)
    for item in tqdm(questions_answers['data'], desc="Answering Questions"):
        question_chunk = item['chunk']
        for sub_item in item['questions']:
            question = sub_item['question']
            context = question_chunk + sub_item['context']
            sub_item['answer'] = gen_answer(client, question, context)
    print('Questions are answered')
    
    with open(questions_answers_file_name, 'w') as json_file:
        json.dump(questions_answers, json_file, indent=4)
    print('saved', questions_answers_file_name)

else:
    with open(questions_answers_file_name, 'r') as json_file:
        questions_answers = json.load(json_file)

    print('loaded', questions_answers_file_name)


top_k context added


Answering Questions: 100%|██████████| 34/34 [43:41<00:00, 77.11s/it] 


Questions are answered
saved ../data/hughes_latex_Q_then_A_use_context/hughes_ch1_QAs_n40_topk10_by_sections.json


## Save a CSV file

In [6]:
"""
Mostafa: 
I think it's better to work with JSON/DataFrame in the code, but for reviewing QAs, CSV is easier to work with
"""

csv_file_name = f"{main_dir}/hughes_ch{chapter}_QAs_n{max_questions}.csv"   
# ----------------------------------

data = []

for item in questions_answers['data']:
    question_chunk = item['chunk']
    for sub_item in item['questions']:
        new_item = {}
        new_item['question_chunk'] = question_chunk
        for k,v in sub_item.items():
            if k == 'embedding':
                continue
            new_item[k] = v
        data.append(new_item)

# data[0]
df = pd.DataFrame(data)[['question_chunk','context','coverage','question','answer']]
df.to_csv(csv_file_name)
df

,question_chunk,context,coverage,question,answer
0,\maketitle,\n\n Additional context 0: \maketitle\n\n Addi...,30,What is the significance of the title in a res...,"Answer: ""NOT ENOUGH INFO."""
1,The main constituents of a finite element meth...,\n\n Additional context 0: The main constituen...,100,What are the main constituents of a finite ele...,The main constituents of a finite element meth...
2,The main constituents of a finite element meth...,\n\n Additional context 0: \begin{enumerate} \...,80,Explain the significance of the variational or...,The variational or weak statement in the finit...
3,The main constituents of a finite element meth...,\n\n Additional context 0: The main constituen...,90,Describe the role of finite element functions ...,Finite element functions play a crucial role i...
4,The main constituents of a finite element meth...,\n\n Additional context 0: The explicit comput...,100,"Given the differential equation $u_{,xx} + f =...","The notation $f: [0,1] \to \mathbb{R}$ indicat..."
...,...,...,...,...,...
336,\begin{enumerate} \setcounter{enumi}{10} \item...,\n\n Additional context 0: i. $A=\left[1-\alph...,30,What are some classical analysis techniques th...,"Answer: ""NOT ENOUGH INFO."""
337,\begin{enumerate} \setcounter{enumi}{12} \item...,\n\n Additional context 0: The main constituen...,30,What are elliptic problems in the context of t...,"Answer: ""NOT ENOUGH INFO."""
338,\begin{enumerate} \setcounter{enumi}{12} \item...,\n\n Additional context 0: \begin{enumerate} \...,30,Discuss the contributions of P. G. Ciarlet to ...,"Answer: ""NOT ENOUGH INFO."""
339,\begin{enumerate} \setcounter{enumi}{12} \item...,\n\n Additional context 0: \begin{enumerate} \...,30,Explain the importance of mathematical theory ...,The importance of mathematical theory in the d...


## Print and review

In [7]:
import textwrap

def print_wrapped(text, wrap_length=160):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [8]:
i = 9  # try different QAs

print('Q:')
print_wrapped(df.iloc[i,:]['question'])
print('A:')
print_wrapped(df.iloc[i,:]['answer'])
print('\nChunk used for Q generation:')
print_wrapped(df.iloc[i,:]['question_chunk'])
print('\nRetrieved context:')
for item in df.iloc[i,:]['context'].split('Additional context'):
    print_wrapped(item)
    print()

Q:
Explain the physical interpretation of the differential equation $u_{,xx} + f = 0$ in terms of the transverse displacement of a string in tension and the
longitudinal displacement of an elastic rod.
A:
The differential equation $u_{,xx} + f = 0$ has significant physical interpretations in the context of mechanics.   1. **Transverse Displacement of a String in
Tension**: In this scenario, the equation describes how the transverse displacement $u$ of a string under tension behaves. The term $u_{,xx}$ represents the
curvature of the string, which is related to how the string bends under the influence of forces. The function $f$ can be interpreted as an external force acting
on the string per unit length. Therefore, the equation states that the curvature of the string (which influences how it bends) is balanced by the external
forces acting on it. If $f$ is positive, it indicates a downward force, causing the string to sag, while if $f$ is negative, it represents an upward force,
which 